In [1]:
import numpy as np
import random

In [14]:
def initialize_agents(grid_size):
    agents = np.zeros(grid_size, dtype=int)
    x, y = random.randint(0, grid_size[0]-1), random.randint(0, grid_size[1]-1)
    agents[x, y] = 1  # Инициализация: один агент знает об инновации
    return agents

def initialize_probabilities():
    return np.array([
        [0.0096, 0.0140, 0.0168, 0.0140, 0.0096],
        [0.0140, 0.301, 0.0547, 0.301, 0.0140],
        [0.0168, 0.0547, 0.4431, 0.0547, 0.0168],
        [0.0140, 0.301, 0.0547, 0.301, 0.0140],
        [0.0096, 0.0140, 0.0168, 0.0140, 0.0096]
    ])

def diffusion_step(agents, probabilities, barriers):
    new_agents = np.copy(agents)
    grid_size = agents.shape

    for x in range(grid_size[0]):
        for y in range(grid_size[1]):
            if agents[x, y] > 0:  # Только для агентов, знающих об инновации
                for r in range(-2, 3):
                    for s in range(-2, 3):
                        nx, ny = x + r, y + s

                        if 0 <= nx < grid_size[0] and 0 <= ny < grid_size[1]:
                            # Применяем неизотропное пространство
                            probability = probabilities[r+2, s+2] * agents[x, y]

                            # Применяем барьеры
                            if barriers[x, y] == 1 and barriers[nx, ny] == 1:
                                probability = 0

                            # Передача инновации с учетом неизотропного пространства
                            if random.uniform(0, 1) < probability:
                                new_agents[nx, ny] += 1

    return new_agents

def print_agents(agents):
    for row in agents:
        print(" ".join([str(agent) for agent in row]))

def main(grid_size, steps):
    agents = initialize_agents(grid_size)
    probabilities = initialize_probabilities()
    barriers = np.zeros(grid_size, dtype=int)
    barriers[grid_size[0]//2, grid_size[1]//2] = 1  # Устанавливаем барьер посередине

    print("Начальное состояние:")
    print_agents(agents)

    for step in range(steps):
        agents = diffusion_step(agents, probabilities, barriers)

    print("\nСостояние после диффузии инноваций:")
    print_agents(agents)

In [15]:
grid_size = (10, 10)
steps = 2
main(grid_size, steps)

Начальное состояние:
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0

Состояние после диффузии инноваций:
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 2 0 0 0 0 0 0 0
3 1 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

# Создаем поле определенного размера
num_iterations = 50
grid_rows = 5
grid_cols = 5
grid_size = (grid_rows, grid_cols)

grid = np.zeros(grid_size)
probabilities = [[0.0096, 0.0140, 0.0168, 0.0140, 0.0096],
                 [0.0096, 0.0301, 0.0547, 0.0301, 0.0096],
                 [0.0168, 0.0547, 0.4431, 0.0547, 0.0168],
                 [0.0096, 0.0301, 0.0547, 0.0301, 0.0096],
                 [0.0096, 0.0140, 0.0168, 0.0140, 0.0096]]

prob_dict = {}
for i in range(grid_rows):
    for j in range(grid_cols):
        prob_dict['{},{}'.format(i, j)] = probabilities[i][j]

# Задаем начальные значения инноваций в центральной клетке на поле
initial_cell = [random.randint(0, grid_rows-1), random.randint(0, grid_cols-1)]

grid[initial_cell[0]][initial_cell[1]] = 1

print(grid)

# Выполняем несколько итераций для распространения инноваций
my_metrics = pd.DataFrame(columns=['period', '%who', 'new'])
for period in range(num_iterations-1):
    new_innovations = 0
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            if grid[i][j] == 0:
                continue
            for n in range(int(grid[i, j])):
                neighbor = random.choices(list(prob_dict.keys()), weights=list(prob_dict.values()), k=1)[0]
                if random.randint(1, N) > grid[int(neighbor.split(',')[0])][int(neighbor.split(',')[1])]:
                    grid[int(neighbor.split(',')[0])][int(neighbor.split(',')[1])] += 1
                    new_innovations += 1
                else:
                    continue
    my_metrics.loc[len(my_metrics)] = [period, round(grid.sum() / grid_cols / grid_rows / N * 100, 2), new_innovations]

    # Визуализируем результаты в виде тепловой карты с числовыми значениями в клетках
    if period % 5 == 0:
        plt.imshow(grid, cmap='hot', interpolation='nearest')
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                plt.annotate(str(grid[i, j]), xy=(j, i), ha='center', va='center')
        plt.colorbar()
        plt.show()